# GRU Model with AutoEncoder

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras import Model
from tensorflow.keras.layers import GRU, LSTM, Dense, Input, Dropout, BatchNormalization, RepeatVector, TimeDistributed, Dense
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split

In [2]:
# Load the dataset
train_x = pd.read_csv('/Users/mohanyang/Documents/GitHub/SUROP_time_series/SUROP/data/train_x.csv')
train_y = pd.read_csv('/Users/mohanyang/Documents/GitHub/SUROP_time_series/SUROP/data/train_y.csv')
test_x = pd.read_csv('/Users/mohanyang/Documents/GitHub/SUROP_time_series/SUROP/data/test_x.csv')

In [3]:
# Display the first few rows
print(train_x.head())
print(train_x.shape)

   Hour  ID    Age  Gender  Unit1  Unit2     HR   O2Sat   Temp     SBP  ...  \
0     1   2  66.67     1.0    0.0    1.0  74.78  100.06  35.61  121.68  ...   
1     2   2  66.67     1.0    0.0    1.0  74.78  100.06  35.61  121.68  ...   
2     3   2  66.67     1.0    0.0    1.0  74.78  100.06  35.61  121.68  ...   
3     4   2  66.67     1.0    0.0    1.0  74.78  100.06  35.61  121.68  ...   
4     5   2  66.67     1.0    0.0    1.0  84.42   99.58  35.61  114.60  ...   

   Phosphate  Potassium  Bilirubin_total  TroponinI    Hct   Hgb  PTT   WBC  \
0        NaN       4.48              NaN        NaN  28.83  9.21  NaN  8.15   
1        NaN       4.48              NaN        NaN  28.83  9.21  NaN  8.15   
2        NaN       4.48              NaN        NaN  28.83  9.21  NaN  8.15   
3        NaN       4.48              NaN        NaN  28.83  9.21  NaN  8.15   
4        NaN       4.48              NaN        NaN  28.83  9.21  NaN  8.15   

   Fibrinogen  Platelets  
0         NaN        Na

In [4]:
print(train_y.head())
print(train_y.shape)

   ID  Outcome
0   2        0
1   3        0
2   5        1
3   6        0
4  13        0
(12115, 2)


In [5]:
max_hours = train_x.groupby('ID')['Hour'].count().max()
print(f'Maximum number of hours: {max_hours}')

Maximum number of hours: 336


In [6]:
# Get unique person IDs
person_ids = train_x['ID'].unique()

# Initialize tensors for features, masks, and time intervals
num_persons = len(person_ids)
num_features = train_x.shape[1] - 2  # excluding 'ID' and 'Hour' columns

In [7]:
person_ids

array([    2,     3,     5, ..., 21630, 21632, 21633])

In [8]:
#Initialize tensors
X_tensor = np.zeros((num_persons, max_hours, num_features+1))

In [9]:
for i, person_id in enumerate(person_ids):
    person_data = train_x[train_x['ID'] == person_id].sort_values('Hour')
    hours = person_data['Hour'].values
    features = person_data.drop(columns=['ID']).values
    
    # Fill X_tensor
    X_tensor[i, :len(hours), :] = features

In [10]:
X_tensor

array([[[  1.  ,  66.67,   1.  , ...,   8.15,    nan,    nan],
        [  2.  ,  66.67,   1.  , ...,   8.15,    nan,    nan],
        [  3.  ,  66.67,   1.  , ...,   8.15,    nan,    nan],
        ...,
        [  0.  ,   0.  ,   0.  , ...,   0.  ,   0.  ,   0.  ],
        [  0.  ,   0.  ,   0.  , ...,   0.  ,   0.  ,   0.  ],
        [  0.  ,   0.  ,   0.  , ...,   0.  ,   0.  ,   0.  ]],

       [[  1.  ,  79.74,   0.  , ...,   6.82, 267.01, 281.18],
        [  2.  ,  79.74,   0.  , ...,   6.82, 267.01, 281.18],
        [  3.  ,  79.74,   0.  , ...,   6.82, 267.01, 281.18],
        ...,
        [  0.  ,   0.  ,   0.  , ...,   0.  ,   0.  ,   0.  ],
        [  0.  ,   0.  ,   0.  , ...,   0.  ,   0.  ,   0.  ],
        [  0.  ,   0.  ,   0.  , ...,   0.  ,   0.  ,   0.  ]],

       [[  1.  ,  60.74,   1.  , ...,  12.45,    nan,  84.77],
        [  2.  ,  60.74,   1.  , ...,  12.45,    nan,  84.77],
        [  3.  ,  60.74,   1.  , ...,  12.45,    nan,  84.77],
        ...,
        [  0

In [11]:
print(X_tensor.shape)

(12115, 336, 39)


In [12]:
# Ensure train_y is aligned with X_tensor
labels = np.zeros(len(person_ids))

for i, person_id in enumerate(person_ids):
    if person_id in train_y['ID'].values:
        labels[i] = train_y[train_y['ID'] == person_id]['Outcome'].values[0]

print(labels.shape)

(12115,)


In [13]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_tensor, labels, test_size=0.2, random_state=10
)

In [14]:
def build_rdae(input_shape):
    inputs = Input(shape=input_shape)
    
    # Encoder
    encoded = LSTM(128, activation='relu')(inputs)
    encoded = RepeatVector(input_shape[0])(encoded)
    
    # Decoder
    decoded = LSTM(128, activation='relu', return_sequences=True)(encoded)
    decoded = TimeDistributed(Dense(input_shape[1]))(decoded)
    
    # Define the model
    autoencoder = Model(inputs, decoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    
    return autoencoder

In [15]:
input_shape = (X_train.shape[1], X_train.shape[2])
rdae = build_rdae(input_shape)

In [16]:
#data with missing values replaced by zero
X_train_filled = np.nan_to_num(X_train)

# Normalize the data to be between 0 and 1
X_train_filled = (X_train_filled - np.min(X_train_filled)) / (np.max(X_train_filled) - np.min(X_train_filled))

In [17]:
# Train the autoencoder
optimizer = Adam()
rdae.compile(optimizer = optimizer, loss='mae')
rdae.fit(X_train_filled, X_train_filled, epochs=20, batch_size=64, validation_split=0.1)

# Use the trained autoencoder to impute missing values
X_train_rdae_imputed = rdae.predict(X_train_filled)

Epoch 1/20


  1/137 ━━━━━━━━━━━━━━━━━━━━ 11:12 5s/step - loss: 0.0289

  2/137 ━━━━━━━━━━━━━━━━━━━━ 1:15 561ms/step - loss: 0.0281

  3/137 ━━━━━━━━━━━━━━━━━━━━ 1:16 574ms/step - loss: 0.0273

  4/137 ━━━━━━━━━━━━━━━━━━━━ 1:18 587ms/step - loss: 0.0266

...

 34/137 ━━━━━━━━━━━━━━━━━━━━ 1:10 685ms/step - loss: 0.0138

 35/137 ━━━━━━━━━━━━━━━━━━━━ 1:09 685ms/step - loss: 0.0136

 36/137 ━━━━━━━━━━━━━━━━━━━━ 1:09 685ms/step - loss: 0.0134

 37/137 ━━━━━━━━━━━━━━━━━━━━ 1:08 685ms/step - loss: 0.0132

KeyboardInterrupt: 

In [317]:
def create_gru_model(input_shape, learning_rate=0.001, dropout_rate=0.2, regularization_factor=0.001):
    x_input = Input(shape=input_shape)
    
    # GRU layers with dropout, batch normalization, and regularization
    gru_output = GRU(units=64, return_sequences=False, kernel_regularizer=l2(regularization_factor))(x_input)
    gru_output = BatchNormalization()(gru_output)
    gru_output = Dropout(dropout_rate)(gru_output)
    
    # Output layer
    output = Dense(1, activation='sigmoid', kernel_regularizer=l2(regularization_factor))(gru_output)
    
    # Create model
    model = Model(inputs=x_input, outputs=output)
    
    # Create an Adam optimizer with a custom learning rate
    optimizer = Adam(learning_rate=learning_rate)
    
    # Compile the model with the custom optimizer
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['AUC'])
    
    return model

In [318]:
input_shape = (max_hours, num_features+1)
model_gru_rdae = create_gru_model(input_shape)

history = model_gru_rdae.fit(X_train_rdae_imputed, y_train, epochs=50, batch_size=16, validation_split=0.2)

Epoch 1/50
485/485 ━━━━━━━━━━━━━━━━━━━━ 24s 45ms/step - AUC: 0.5017 - loss: 0.5034 - val_AUC: 0.5000 - val_loss: 0.4163
Epoch 2/50
485/485 ━━━━━━━━━━━━━━━━━━━━ 22s 44ms/step - AUC: 0.4952 - loss: 0.3927 - val_AUC: 0.5000 - val_loss: 0.4164
Epoch 3/50
485/485 ━━━━━━━━━━━━━━━━━━━━ 21s 44ms/step - AUC: 0.4638 - loss: 0.4026 - val_AUC: 0.4997 - val_loss: 0.4137
Epoch 4/50
485/485 ━━━━━━━━━━━━━━━━━━━━ 21s 44ms/step - AUC: 0.4756 - loss: 0.3977 - val_AUC: 0.5000 - val_loss: 0.4156
Epoch 5/50
485/485 ━━━━━━━━━━━━━━━━━━━━ 21s 44ms/step - AUC: 0.5079 - loss: 0.4068 - val_AUC: 0.5000 - val_loss: 0.4261
Epoch 6/50
485/485 ━━━━━━━━━━━━━━━━━━━━ 22s 45ms/step - AUC: 0.4770 - loss: 0.4028 - val_AUC: 0.5000 - val_loss: 0.4167
Epoch 7/50
485/485 ━━━━━━━━━━━━━━━━━━━━ 21s 44ms/step - AUC: 0.5100 - loss: 0.3867 - val_AUC: 0.5000 - val_loss: 0.4127
Epoch 8/50
485/485 ━━━━━━━━━━━━━━━━━━━━ 21s 44ms/step - AUC: 0.4911 - loss: 0.3979 - val_AUC: 0.5000 - val_loss: 0.4237
Epoch 9/50
485/485 ━━━━━━━━━━━━━━━━━━━━ 

KeyboardInterrupt: 

In [ ]:
# Evaluate the model on the test set
test_loss, test_auc = model_gru_rdae.evaluate(X_test, y_test)
print(f'Test AUC: {test_auc}')

In [ ]:
# Plot training & validation AUC values
plt.plot(history.history['AUC'])
plt.plot(history.history['val_AUC'])
plt.title('Model AUC')
plt.ylabel('AUC')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()